In [1]:
from keras.models import Sequential
from keras.layers import BatchNormalization, Dropout, Dense, Flatten, Reshape, LeakyReLU, Conv2D, Conv2DTranspose
from tensorflow.keras.optimizers import Adam
import numpy as np

In [3]:
from keras.utils import load_img, img_to_array

In [4]:
!mkdir gen_my_imgs

In [5]:
images_path = "my_imgs"

In [6]:
import os
images= []
for image in os.listdir(images_path):
    images.append(img_to_array(load_img(images_path +'/'+ image,target_size=(32,32))))
    

In [7]:
img_width= 32
img_height= 32
channels= 3
img_shape= (img_width, img_height, channels)
latent_dim= 100
adam= Adam(learning_rate= 0.0002)

In [9]:
def build_generator():
    model = Sequential()
    model.add(Dense(256 * 4 * 4, input_dim= latent_dim))
    model.add(LeakyReLU(alpha= 0.2))
    model.add(Reshape((4, 4, 256)))
    model.add(Conv2DTranspose(128, (4,4) , strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha= 0.2))
    model.add(Conv2DTranspose(128, (4,4) , strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha= 0.2))
    model.add(Conv2DTranspose(128, (4,4) , strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha= 0.2))
    model.add(Conv2D(3, (3,3), activation= 'tanh', padding= 'same'))
    
    model.summary()
    return model

generator= build_generator()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 4096)              413696    
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 4096)              0         
                                                                 
 reshape_1 (Reshape)         (None, 4, 4, 256)         0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 8, 8, 128)        524416    
 nspose)                                                         
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 8, 8, 128)         0         
                                                                 
 conv2d_transpose_1 (Conv2DT  (None, 16, 16, 128)      262272    
 ranspose)                                            

In [10]:
def build_discriminator():
    model = Sequential()
    model.add(Conv2D(64, (3,3), padding='same', input_shape= img_shape))
    model.add(LeakyReLU(0.2))
    model.add(Conv2D(128, (3,3), padding='same'))
    model.add(LeakyReLU(0.2))
    model.add(Conv2D(128, (3,3), padding='same'))
    model.add(LeakyReLU(0.2))
    model.add(Conv2D(256, (3,3), padding='same'))
    model.add(LeakyReLU(0.2))

    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation= 'sigmoid'))    
    
    model. summary()
    return model

discriminator= build_discriminator()
discriminator.compile(optimizer= 'adam', loss= 'binary_crossentropy', metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 32, 32, 64)        1792      
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 32, 32, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 32, 128)       73856     
                                                                 
 leaky_re_lu_6 (LeakyReLU)   (None, 32, 32, 128)       0         
                                                                 
 conv2d_3 (Conv2D)           (None, 32, 32, 128)       147584    
                                                                 
 leaky_re_lu_7 (LeakyReLU)   (None, 32, 32, 128)       0         
                                                                 
 conv2d_4 (Conv2D)           (None, 32, 32, 256)      

In [11]:

gan = Sequential()
gan.add(generator)
discriminator.trainable= False
gan.add(discriminator)

gan.compile(optimizer= 'adam', loss= 'binary_crossentropy', metrics=['accuracy'])

In [12]:
import matplotlib.pyplot as plt
import glob
import imageio
import PIL

save_name= 0.00000000

def save_image(epoch):
    r, c = 5, 5 
    noise= np.random.normal(0,1, (r*c, latent_dim))
    gen_imgs= generator.predict(noise)
    global save_name
    save_name+= 0.00000001
    print("%.8f" % save_name)
    gen_imgs= 0.5 * gen_imgs + 0.5
    fig, axs= plt.subplots(r,c)
    cnt= 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt,:,:,0])
            axs[i,j].axis('off')
            cnt+=1
    fig.savefig("gen_my_imgs/%.8f.png" % save_name)
    plt.close()


Matplotlib is building the font cache; this may take a moment.


In [15]:
def train(epochs, batch_size= 64, save_interval= 200):
    x= np.array(images)
    x= x/ 127.5 -1.
    valid= np.ones((batch_size, 1))
    fake= np.zeros((batch_size, 1))
    batch_per_epoch= int(x.shape[0] /batch_size)
    
    for epoch in range(epochs):
        for j in range(batch_per_epoch):
            idx= np.random.randint(0, x.shape[0], batch_size)
            imgs= x[idx]

            noise= np.random.normal(0,1, (batch_size, latent_dim))
            gen_imgs= generator.predict(noise)

            # train discriminator
            d_loss_real= discriminator.train_on_batch(imgs, valid)
            d_loss_fake= discriminator.train_on_batch(gen_imgs, fake)
            d_loss= np.add(d_loss_real,d_loss_fake) * 0.5

            noise= np.random.normal(0, 1, (batch_size, latent_dim))
            g_loss= gan.train_on_batch(noise, valid)

            print(f'{epoch} [D loss {d_loss} ] [G loss {g_loss}]')
        
        save_image(epoch)
        
train(30000,8)

1/1 [==============================] - 1s 853ms/step
0 [D loss [0.74697268 0.3125    ] ] [G loss [0.7721751928329468, 0.0]]
1/1 [==============================] - 0s 151ms/step
0 [D loss [0.60809495 0.5       ] ] [G loss [1.2422642707824707, 0.0]]
1/1 [==============================] - 0s 145ms/step
0 [D loss [0.22005182 1.        ] ] [G loss [1.4344466924667358, 0.0]]
1/1 [==============================] - 0s 154ms/step
0 [D loss [0.11715746 1.        ] ] [G loss [2.079291582107544, 0.0]]
1/1 [==============================] - 0s 168ms/step
0 [D loss [0.06537765 1.        ] ] [G loss [3.2221755981445312, 0.0]]
1/1 [==============================] - 0s 193ms/step
0 [D loss [0.07766093 1.        ] ] [G loss [5.786121845245361, 0.0]]
1/1 [==============================] - 0s 128ms/step
0 [D loss [2.34289215e-06 1.00000000e+00] ] [G loss [19.260059356689453, 0.0]]
1/1 [==============================] - 0s 139ms/step
0 [D loss [3.44651135 0.875     ] ] [G loss [6.905327796936035, 0.0]]
1/1

1/1 [==============================] - 0s 162ms/step
2 [D loss [474.82879639   0.875     ] ] [G loss [696.84716796875, 0.0]]
1/1 [==============================] - 0s 158ms/step
2 [D loss [0. 1.] ] [G loss [518.1009521484375, 0.0]]
1/1 [==============================] - 0s 160ms/step
2 [D loss [0. 1.] ] [G loss [317.61712646484375, 0.0]]
1/1 [==============================] - 0s 162ms/step
2 [D loss [28.43784809  0.5625    ] ] [G loss [961.9369506835938, 0.0]]
1/1 [==============================] - 0s 196ms/step
2 [D loss [128.40112305   0.625     ] ] [G loss [1369.8546142578125, 0.0]]
1/1 [==============================] - 0s 146ms/step
2 [D loss [17.5333786  0.875    ] ] [G loss [1831.28662109375, 0.0]]
1/1 [==============================] - 0s 143ms/step
2 [D loss [17.48916699  0.8125    ] ] [G loss [2051.8466796875, 0.0]]
1/1 [==============================] - 0s 158ms/step
2 [D loss [19.30739021  0.8125    ] ] [G loss [2427.5244140625, 0.0]]
1/1 [==============================] - 

1/1 [==============================] - 0s 144ms/step
4 [D loss [3.88040693e-07 1.00000000e+00] ] [G loss [81.75931549072266, 0.0]]
1/1 [==============================] - 0s 142ms/step
4 [D loss [1.29325879 0.9375    ] ] [G loss [60.64818572998047, 0.125]]
1/1 [==============================] - 0s 146ms/step
4 [D loss [7.00066125e-04 1.00000000e+00] ] [G loss [65.88818359375, 0.0]]
1/1 [==============================] - 0s 135ms/step
4 [D loss [1.12277894e-06 1.00000000e+00] ] [G loss [76.33076477050781, 0.0]]
1/1 [==============================] - 0s 330ms/step
0.00000005
1/1 [==============================] - 0s 167ms/step
5 [D loss [0.01499697 1.        ] ] [G loss [67.62548828125, 0.0]]
1/1 [==============================] - 0s 150ms/step
5 [D loss [1.05104207e-21 1.00000000e+00] ] [G loss [67.38618469238281, 0.0]]
1/1 [==============================] - 0s 137ms/step
5 [D loss [3.18257763e-09 1.00000000e+00] ] [G loss [59.3915901184082, 0.125]]
1/1 [==============================] -

1/1 [==============================] - 0s 133ms/step
7 [D loss [7.42468067e-07 1.00000000e+00] ] [G loss [33.65062713623047, 0.0]]
1/1 [==============================] - 0s 144ms/step
7 [D loss [1.21399782e-11 1.00000000e+00] ] [G loss [21.461410522460938, 0.0]]
1/1 [==============================] - 0s 143ms/step
7 [D loss [4.53217487e-13 1.00000000e+00] ] [G loss [21.74640655517578, 0.125]]
1/1 [==============================] - 0s 138ms/step
7 [D loss [5.28723683e-09 1.00000000e+00] ] [G loss [32.448829650878906, 0.0]]
1/1 [==============================] - 0s 134ms/step
7 [D loss [0.09839953 0.9375    ] ] [G loss [31.01952362060547, 0.0]]
1/1 [==============================] - 0s 139ms/step
7 [D loss [8.16882774e-04 1.00000000e+00] ] [G loss [31.81383514404297, 0.0]]
1/1 [==============================] - 0s 149ms/step
7 [D loss [2.54624954e-06 1.00000000e+00] ] [G loss [19.649967193603516, 0.125]]
1/1 [==============================] - 0s 131ms/step
7 [D loss [0.05822421 0.9375   

1/1 [==============================] - 0s 131ms/step
9 [D loss [5.5263401e-04 1.0000000e+00] ] [G loss [48.58042907714844, 0.0]]
1/1 [==============================] - 0s 135ms/step
9 [D loss [5.85740468e-10 1.00000000e+00] ] [G loss [39.62248229980469, 0.0]]
1/1 [==============================] - 0s 134ms/step
9 [D loss [0.00240022 1.        ] ] [G loss [36.930702209472656, 0.0]]
1/1 [==============================] - 0s 134ms/step
9 [D loss [5.8365222e-05 1.0000000e+00] ] [G loss [39.549442291259766, 0.0]]
1/1 [==============================] - 0s 140ms/step
9 [D loss [3.38708717e-12 1.00000000e+00] ] [G loss [36.50223159790039, 0.0]]
1/1 [==============================] - 0s 138ms/step
9 [D loss [8.62181848e-09 1.00000000e+00] ] [G loss [41.554744720458984, 0.0]]
1/1 [==============================] - 0s 138ms/step
9 [D loss [2.78387291e-07 1.00000000e+00] ] [G loss [38.64522171020508, 0.0]]
1/1 [==============================] - 0s 151ms/step
9 [D loss [0.00109207 1.        ] ] [G 

...

1/1 [==============================] - 0s 141ms/step
18 [D loss [8.7883659e-06 1.0000000e+00] ] [G loss [23.110000610351562, 0.0]]
1/1 [==============================] - 0s 130ms/step
18 [D loss [3.70711774e-04 1.00000000e+00] ] [G loss [18.71822166442871, 0.0]]
1/1 [==============================] - 0s 135ms/step
18 [D loss [1.437838e-04 1.000000e+00] ] [G loss [22.857046127319336, 0.0]]
1/1 [==============================] - 0s 145ms/step
18 [D loss [0.00169346 1.        ] ] [G loss [22.091522216796875, 0.0]]
1/1 [==============================] - 0s 137ms/step
18 [D loss [2.50737875e-05 1.00000000e+00] ] [G loss [28.401275634765625, 0.0]]
1/1 [==============================] - 0s 341ms/step
0.00000019
1/1 [==============================] - 0s 166ms/step
19 [D loss [9.02432909e-05 1.00000000e+00] ] [G loss [20.161958694458008, 0.0]]
1/1 [==============================] - 0s 137ms/step
19 [D loss [0.00423696 1.        ] ] [G loss [24.513153076171875, 0.0]]
1/1 [=======================

21 [D loss [3.65368418e-09 1.00000000e+00] ] [G loss [28.83819580078125, 0.0]]
1/1 [==============================] - 0s 157ms/step
21 [D loss [1.09962317e-04 1.00000000e+00] ] [G loss [29.643043518066406, 0.0]]
1/1 [==============================] - 0s 210ms/step
21 [D loss [4.70097448e-10 1.00000000e+00] ] [G loss [31.13500213623047, 0.0]]
1/1 [==============================] - 0s 215ms/step
21 [D loss [6.27552605e-05 1.00000000e+00] ] [G loss [26.967252731323242, 0.0]]
1/1 [==============================] - 0s 188ms/step
21 [D loss [0.48114157 0.9375    ] ] [G loss [37.865234375, 0.0]]
1/1 [==============================] - 0s 176ms/step
21 [D loss [2.22603722e-04 1.00000000e+00] ] [G loss [28.868242263793945, 0.0]]
1/1 [==============================] - 0s 139ms/step
21 [D loss [1.40127947e-04 1.00000000e+00] ] [G loss [25.93536376953125, 0.0]]
1/1 [==============================] - 0s 132ms/step
21 [D loss [0.20216389 0.9375    ] ] [G loss [21.387929916381836, 0.0]]
1/1 [=========

1/1 [==============================] - 0s 155ms/step
23 [D loss [5.51411475e-06 1.00000000e+00] ] [G loss [16.021121978759766, 0.0]]
1/1 [==============================] - 0s 181ms/step
23 [D loss [6.41538336e-05 1.00000000e+00] ] [G loss [15.291688919067383, 0.0]]
1/1 [==============================] - 0s 154ms/step
23 [D loss [0.00192069 1.        ] ] [G loss [20.18518829345703, 0.0]]
1/1 [==============================] - 0s 185ms/step
23 [D loss [1.5891143e-05 1.0000000e+00] ] [G loss [18.391071319580078, 0.0]]
1/1 [==============================] - 0s 200ms/step
23 [D loss [7.32478163e-05 1.00000000e+00] ] [G loss [19.115985870361328, 0.0]]
1/1 [==============================] - 0s 180ms/step
23 [D loss [3.9583388e-05 1.0000000e+00] ] [G loss [18.289974212646484, 0.0]]
1/1 [==============================] - 0s 140ms/step
23 [D loss [4.14024691e-04 1.00000000e+00] ] [G loss [20.2144832611084, 0.0]]
1/1 [==============================] - 0s 143ms/step
23 [D loss [5.79488438e-06 1.00

1/1 [==============================] - 0s 158ms/step
25 [D loss [1.25215941e-04 1.00000000e+00] ] [G loss [26.662593841552734, 0.0]]
1/1 [==============================] - 0s 142ms/step
25 [D loss [1.64714118e-07 1.00000000e+00] ] [G loss [25.839794158935547, 0.0]]
1/1 [==============================] - 0s 138ms/step
25 [D loss [0.00272218 1.        ] ] [G loss [23.998409271240234, 0.0]]
1/1 [==============================] - 0s 134ms/step
25 [D loss [0.00467854 1.        ] ] [G loss [28.09551239013672, 0.0]]
1/1 [==============================] - 0s 139ms/step
25 [D loss [1.61264364e-05 1.00000000e+00] ] [G loss [30.301692962646484, 0.0]]
1/1 [==============================] - 0s 148ms/step
25 [D loss [2.26171046e-08 1.00000000e+00] ] [G loss [32.10635757446289, 0.0]]
1/1 [==============================] - 0s 183ms/step
25 [D loss [2.24557575e-06 1.00000000e+00] ] [G loss [28.102890014648438, 0.0]]
1/1 [==============================] - 0s 171ms/step
25 [D loss [1.01928261e-07 1.00000

KeyboardInterrupt: 

In [ ]:
anim_file = 'dcgan.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = glob.glob('gen_my_imgs/*.png')
  filenames = sorted(filenames)
  for filename in filenames:
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)

In [ ]:
noise = np.random.normal(0, 1, (16, latent_dim))
gen_imgs = generator.predict(noise)
gen_imgs = (gen_imgs + 1) / 2.0
plt.imshow(gen_imgs[2])

In [ ]:
generator.save_weights("/gen_bobross_images/generator1hour.h5")
discriminator.save_weights("/gen_bobross_images/discriminator1hour.h5")